# Scipy - scientific library for Python


`scipy` is buildt on top of the `numpy` framework for multidimensional arrays, and provides a large number of higher-level scientific algorithms. Some of the topics that SciPy covers are:

* Special functions ([scipy.special](http://docs.scipy.org/doc/scipy/reference/special.html))
* Integration ([scipy.integrate](http://docs.scipy.org/doc/scipy/reference/integrate.html))
* Optimization ([scipy.optimize](http://docs.scipy.org/doc/scipy/reference/optimize.html))
* Interpolation ([scipy.interpolate](http://docs.scipy.org/doc/scipy/reference/interpolate.html))
* Fourier Transforms ([scipy.fftpack](http://docs.scipy.org/doc/scipy/reference/fftpack.html))
* Signal Processing ([scipy.signal](http://docs.scipy.org/doc/scipy/reference/signal.html))
* Linear Algebra ([scipy.linalg](http://docs.scipy.org/doc/scipy/reference/linalg.html))
* Sparse Eigenvalue Problems ([scipy.sparse](http://docs.scipy.org/doc/scipy/reference/sparse.html))
* Statistics ([scipy.stats](http://docs.scipy.org/doc/scipy/reference/stats.html))
* Multi-dimensional image processing ([scipy.ndimage](http://docs.scipy.org/doc/scipy/reference/ndimage.html))
* File IO ([scipy.io](http://docs.scipy.org/doc/scipy/reference/io.html))

Each of these submodules provides a number of functions and classes that can be used to solve problems in their respective topics.


## Special functions
Scipy implements a large amount of *special functions* (Bessel function,
Airy function, orthogonal polynomials, ...) for numneric calculations. They can be used as functions within `numpy`.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import scipy.special as ss

# we plot the n\in [1..3] Legendre polynomials.
#
x = np.linspace(-1.0, 1.0, 100)

for n in range(1, 4):
    y = ss.eval_legendre(n, x)
    plt.plot(x, y, label=r"$L_%d(x)$" % n)
    
plt.legend()    

## Numerical Integration

Numerical evaluation of a function of the type

$\displaystyle \int_a^b f(x)\, \text{d}x$

is called *numerical integration*, or *quadature*.
`Scipy` provides many funtions for quadrature, for example the `quad` and `dblquad` for single and double integrals, respectively.

In the follwoing I show simple examples to demonstrate the basic usage:

In [ ]:
import scipy
import scipy.integrate as si
import numpy as np

# A normalised Gauss function
def gauss(x):
    factor = (1.0 / np.sqrt(2.0 * np.pi))
    expon = np.exp(-(x)**2 / 2.0)
    
    return factor * expon

# lower and upper integration limits:
x_lower = 0 # the lower limit of x
x_upper = 1 # the upper limit of x

result, abserr = si.quad(gauss, x_lower, x_upper)

print(f"integral value = {result}; absolute error = {abserr}")

# Also infinite limits are possible:
result, abserr = si.quad(gauss, -scipy.Inf, scipy.Inf)

print(f"integral value = {result}; absolute error = {abserr}")


For simple functions we can use a lambda expression:

In [ ]:
import scipy.integrate as si
import numpy as np

val, abserr = si.quad(lambda x: (np.sin(x) + np.cos(x)**2),
                      0, np.pi / 2.)

print(f"result  = {val} +/- {abserr}")


Two-dimensional integrals work similarily

In [ ]:
import scipy.integrate as si
import numpy as np

def integrand(x, y):
    return np.exp(-x**2 - y**2)

x_lower = 0  
x_upper = 10
y_lower = 0
y_upper = 10

# note that the y-limits depend on x in general;
# hence they need to be given as functions of x!
val, abserr = si.dblquad(integrand, x_lower, x_upper, lambda x : y_lower, lambda x: y_upper)

print(val, abserr) 

## Optimization / Fitting

Determining a parametric model $y = m(x; a_0, a_1, \dots a_n)$, ($a_i$ are unknown parameters) to given data points
$(x_i; y_i \pm \Delta y_i);\; i\in [1, \dots, m]$ is called *data-fitting*. Usually the measurements $y_i$ come with some errors $\Delta y_i$. `Scipy` offers several functions for data fitting and I will show you the simplest one: `curve_fit`. It determines the best fit parameters with the $\chi^2$-method, i.e. it determines best fit parameters by minimizing the expression:

$$
\chi^2 = \sum_{i=1}^n\frac{(y_i-m(x_i; a_0, a_1, \dots a_n))^2}{(\Delta y_i)^2}
$$

Please read the [curve_fit documentation](http://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.curve_fit.html#scipy.optimize.curve_fit) on details (error handling etc.).

<img src="figs/chi2.png" style="height: 300px;">

For demonstration purposes we perform a line fit on some fake data:

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import numpy.random as nr
import scipy.optimize as so

# create some fake data and plot them:
numpoints = 10 # the number of points we want to simulate
x = np.random.uniform(0., 100., numpoints)
# The error on each point comes from a normal distribution
# with sigma = 10
y = 2. * x + 2.6 + np.random.normal(0., 10., numpoints)
plt.errorbar(x, y, yerr=10, fmt=".")

In [ ]:
def fit_line(x, a, b):
    return a * x + b

# now perform the fit
# Please read carefully the documentation to see how errors
# are handled. In Physics we typically give absolute errors,
# not relative ones!
popt, pcov = so.curve_fit(fit_line, x, y,
                          sigma = np.ones(y.shape) * 10, absolute_sigma=True)
print(popt)
print(pcov)
print(f"a = {popt[0]} +/- {np.sqrt(pcov[0][0])}")
print(f"b = {popt[1]} +/- {np.sqrt(pcov[1][1])}")

x_fit = np.linspace(0.0, 100, 100)
y_fit = fit_line(x_fit, *(popt))
plt.errorbar(x, y, yerr=10, fmt=".")
plt.plot(x_fit, y_fit, 'r-')

### Careful when the fit parameters seem fine but not the errors

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import numpy.random as nr
import scipy.optimize as so

def fit_func(x, a, b):
    return a * np.exp(x + b)

numpoints = 10 # the number of points we want to simulate
x = np.random.uniform(-2., 2., numpoints)
y = 0.2 * np.exp(x + 5) + np.random.normal(0., 10., numpoints)
plt.errorbar(x, y, yerr=10, fmt=".")

popt, pcov = so.curve_fit(fit_func, x, y,
                          sigma = np.ones(y.shape) * 10, absolute_sigma=True)
print(popt)
print(pcov)
x_fit = np.linspace(-2.0, 2.0, 20)
plt.plot(x_fit, fit_func(x_fit, *(popt)), 'r-')

### Warning: Do not use `curve_fit` if you have errors in both variables

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import numpy.random as nr
import scipy.optimize as so

def fit_line(x, a, b):
    return a * x + b

# create some fake data with errors in 'both' coordinates and plot them:
numpoints = 50 # the number of points we want to simulate
x = np.random.uniform(0., 100., numpoints)
y = x

# add errors in x and y
x = x + np.random.normal(0., 10., numpoints)
y = y + np.random.normal(0., 10., numpoints)

plt.errorbar(x, y, xerr=10, yerr=10, fmt=".")

# perform a fit as we did before (neglecting errors in x!):
popt, pcov = so.curve_fit(fit_line, x, y,
                          sigma = np.ones(y.shape) * 10, absolute_sigma=True)
plt.plot(x, fit_line(x, *(popt)), 'r-')
print(popt)

# now change the roles of x and y and fit again:
x1, y1 = y, x
popt, pcov = so.curve_fit(fit_line, x1, y1,
                          sigma = np.ones(y.shape) * 10, absolute_sigma=True)
plt.plot(x1, fit_line(x1, *(popt)), 'g-')
print(popt)
# Note that you do not get the 'inverse' of the first fit as you might expect!

For fits with errors in $x$ and $y$ you have to use other methods such as *Orthogonal distance regression*. I give you an example script with [chi2FitXYErr.py](code/chi2FitXYErr.py).

In [ ]:
!cat ./data/dataxy.txt
!cat ./data/dataxy_reversed.txt

In [ ]:
%run ./code/chi2FitXYErr.py -i ./data/dataxy.txt

In [ ]:
# The fit-result from the reversed data set also gives a 'inverse' line!
%run ./code/chi2FitXYErr.py -i ./data/dataxy_reversed.txt

## Ordinary Differential Equations
`scipy` offers several functions to *numerically* solve Ordinary Differential Equations (ODEs). We look more closely at `odeint` within the `scipy.integrate`-module.

# First Order ODEs

Most ODEs in physics are of second order. However, most numeric ODE solvers *only* can deal with first order equations. 

We start with:

$$ \frac{dx}{dt} -xt = 0 \hspace{10mm} x(0) = 0.5$$

The first thing we need to do is write it in the form

$$\frac{dx}{dt} = \dot{x} = f(t,x)$$

In other words, "derivative of x equals something that depends on $x$ and time". This is easy in this example:

$$\frac{dx}{dt} = xt.$$

<img src="figs/dgl_field.png" style="height: 300px;">

To numerically solve the ODE, we need to write it first in **python form**:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.integrate as si

In [ ]:
def dxdt(x, t):
    return x * t

x0 = 0.5

Solve differential equation. There are two main solvers in scipy

* `odeint`: Pretty classic, uses a particular solve called lsoda from the FORTRAN library odepack
* `solve_ivp`: More customizable, can choose from a list of possible solvers

Please have a closer look at `solve_ivp` when your fields requires solving complex ODEs. For *simple* cases of our course, `odeint` is sufficient.

In [ ]:
t = np.linspace(0, 2, 50)

sol = si.odeint(dxdt, y0=x0, t=t)

Look at the solution. It will become obvious why it is returned in this form once we deal with systems of ODEs (what these solvers are really meant for)

In [ ]:
sol

Plot

In [ ]:
plt.plot(t, sol[:,0])
plt.ylabel('$x(t)$', fontsize=15)
plt.xlabel('$t$', fontsize=15)

# Coupled first order ODEs

$$ \dot{x}_1 = x_1 + x_2^2 + 3t \hspace{10mm} x_1(0)=0$$
$$ \dot{x}_2 = 3x_1 + x_2^3 - \cos(t) \hspace{10mm} x_2(0)=0$$

Letting $S=(x_1, x_2)$ we need to write a function that returns $dS/dt = (dx_1/dt, dx_2/dt)$. The function $dS/dt$ can take in $S=(x_1, x_2)$ and $t$. This is like before, but in vector format

$$ \vec{S} = \begin{bmatrix} x_1 \\ x_2 \end{bmatrix} \hspace{10mm} \implies  \hspace{10mm} \frac{d\vec{S}}{dt} = \vec{f}(\vec{S}, t) = \vec{f}(x_1, x_2, t) =  \begin{bmatrix} \dot{x}_1 \\ \dot{x}_2 \end{bmatrix}  = \begin{bmatrix}  x_1 + x_2^2 + 3t\\ 3x_1 + x_2^3 - \cos(t) \end{bmatrix}$$

In [ ]:
def dSdt(S, t):
    x1, x2 = S
    return (x1 + x2**2  + 3*t,
            3*x1 + x2**3 - np.cos(t))
x1_0 = 0
x2_0 = 0
S_0 = (x1_0, x2_0)

Solve this ODE

In [ ]:
t = np.linspace(0, 1, 100)
sol = si.odeint(dSdt, y0=S_0, t=t)

Plot $x_1$ and $x_2$ from the solution

In [ ]:
plt.plot(t, sol[:,0], label="$x_1$")
plt.plot(t, sol[:,1], label="$x_2$")
plt.legend()

# Second Order ODEs

**Python does not have functions to directly solve second order ODEs**. 
* But **any second order ODE can be converted into two first order ODEs**.

Consider 

$$\ddot{x} = -\dot{x}^2 + \sin(x)$$

We can convert this into two first order ODEs as follows:

* Take $x$ (this is what we're trying to solve for). Then define $\dot{x}=v$ so that $v$ becomes a new variable.
* Note that $\dot{x}=v$ is one differential equation of first order.
* Since $\dot{v} = \ddot{x} = -\dot{x}^2 + \sin(x) = -v^2 + \sin(x)$ we get another differential equation of first order.

Our two equations:

$$\dot{x} = v$$
$$\dot{v} = -v^2 + \sin(x)$$

These are two coupled first order equations. They require an initial condition ($x_0$ and $v_0$)

$$ \vec{S} = \begin{bmatrix} x \\ v \end{bmatrix} \hspace{10mm} \implies  \hspace{10mm} \frac{d\vec{S}}{dt} = \vec{f}(\vec{S}, t) = \vec{f}(x, v, t) =  \begin{bmatrix} x' \\ v' \end{bmatrix}  = \begin{bmatrix}  v\\ -v^2 + \sin(x)\end{bmatrix}$$

In [ ]:
def dSdt(S, t):
    x, v = S
    return (v,
           -v**2 + np.sin(x))
x_0 = 0
v_0 = 5
S_0 = (x_0, v_0)

In [ ]:
t = np.linspace(0, 1, 100)
sol = si.odeint(dSdt, y0=S_0, t=t)

In [ ]:
plt.plot(t, sol[:,0], label="x")
plt.plot(t, sol[:,1], label="$v = \dot{x}$")
plt.legend()

This process can be repeated for third order (required defining two new variables) fourth order (requires defining three new variables) and so on...

**Example:** Two coupled third order equations

$$\dddot{x_2}= -\ddot{x_1}^3 + \dot{x_2} + x_1 + \sin(t)$$
$$\dddot{x_1}= -2\dot{x_2}^2 + x_2$$

Define 
* $v_1 = \dot{x_1}$
* $v_2 = \dot{x_2}$
* $w_1 = \ddot{x_1} = \dot{v_1}$
* $w_2 = \ddot{x_2} = \dot{v_2}$

These make up four differential equations. Then noting that $\dot{w_1} = \dddot{x_1}$ and $\dot{w_2} = \dddot{x_2}$ we get

* $\dot{w_2} = -w_1^3 + v_2 + x_1 + \sin(t)$
* $\dot{w_1} = -2v_2^2 + x_2$

Then

$$\vec{S} = \begin{bmatrix} x_1\\ v_1 \\ w_1 \\ x_2 \\ v_2 \\ w_2 \end{bmatrix} \hspace{10mm} \implies \hspace{10mm} \frac{d\vec{S}}{dt} = \begin{bmatrix} \dot{x_1}\\ \dot{v_1} \\ \dot{w_1} \\ \dot{x_2} \\ \dot{v_2} \\ \dot{w_2} \end{bmatrix} =  \begin{bmatrix} v_1\\ w_1 \\ -2v_2^2 + x_2 \\ v_2 \\ w_2 \\ -w_1^3 + v_2 + x_1 + \sin(t) \end{bmatrix}$$

In [ ]:
def dSdt(S, t):
    x1, v1, w1, x2, v2, w2 = S
    return [v1,
            w1,
            -2*v2**2 + x2,
            v2,
            w2,
            -w1**3 + v2 + v1 + np.sin(t)]
x1_0 = 0
v1_0 = 0
w1_0 = 0
x2_0 = 0
v2_0 = 0
w2_0 = 0
v_0 = 0
S_0 = (x1_0, v1_0, w1_0, x2_0, v2_0, w2_0)

In [ ]:
t = np.linspace(0, 1, 100)
sol = si.odeint(dSdt, y0=S_0, t=t)

In [ ]:
plt.plot(t,sol[:,0], label="x1")
plt.plot(t,sol[:,3], label="x2")
plt.legend()